In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import operator
import time, datetime
from utils import *

from IPython.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

# import plotly.express as px
# from plotly.subplots import make_subplots
# import plotly.graph_objs as go 

from tqdm import tqdm

# 1. statistics overview

In [2]:
# # There are 784309 tables in our data set. 
# Each table describes a student’s question-solving log. 
# There is no difference in the information dimension between the tables.
# Each table contains the timestamp,solving_id,question_id,user_answer and elapsed_time 
# as described in the above Columns Description section.

In [3]:
# A major property of EdNet is that the questions come in bundles. 
# That is, a collection of questions sharing a common passage, 
# picture or listening material. 
# For example, questions of ID q2319, q2320 and q2321 may share the same reading passage. 
# In this case, the questions are said to form a bundle and will be given to the student 
# with corresponding shared material. 
# When a bundle is given, a student have access to all the problems and has to respond 
# all of them in order to complete the bundle.

In [4]:
import os
path='/mnt/qb/work/mlcolab/hzhou52/kt/EdNet/KT3'
d=[]
table_list=[]
s=pd.Series(os.listdir(path))

In [5]:
len(s)

297915

In [7]:
file_selected=s#s.sample(5000).to_numpy()
for file_name in file_selected:
    data_raw=pd.read_csv(os.path.join(path,file_name),encoding = "ISO-8859-15")
    data_raw['user_id']=pd.Series([file_name[:-4]]*len(data_raw))
    d.append([file_name[:-4],len(data_raw)])
    data=pd.DataFrame(data_raw,columns=['user_id']+data_raw.columns.to_list()[:-1])
    table_list.append(data)
df=pd.concat(table_list)
pd.set_option('display.max_rows',10)
df=df.reset_index(drop=True)
df

,user_id,timestamp,action_type,item_id,source,user_answer,platform
0,u1,1565096151269,enter,b3544,diagnosis,NaN,mobile
1,u1,1565096187972,respond,q5012,diagnosis,b,mobile
2,u1,1565096194904,submit,b3544,diagnosis,NaN,mobile
3,u1,1565096195001,enter,b3238,diagnosis,NaN,mobile
4,u1,1565096218682,respond,q4706,diagnosis,c,mobile
...,...,...,...,...,...,...,...
92094811,u9998,1568964975390,enter,b3819,sprint,NaN,mobile
92094812,u9998,1568964992921,respond,q5287,sprint,c,mobile
92094813,u9998,1568964996503,submit,b3819,sprint,NaN,mobile
92094814,u9998,1568964996572,enter,e3819,sprint,NaN,mobile


In [18]:
import pickle

filehandler = open("/mnt/qb/work/mlcolab/hzhou52/kt/EdNet/kt3.obj","wb")
pickle.dump(df, filehandler)
filehandler.close()

# file = open("Fruits.obj",'rb')
# object_file = pickle.load(file)
# file.close()

In [8]:
log = df
print('The columns of log are ', log.columns)
print('The length of log is {}'.format(len(log)))

print('###########################################')

print(count_unique(log, log.columns))

The columns of log are  Index(['user_id', 'timestamp', 'action_type', 'item_id', 'source',
       'user_answer', 'platform'],
      dtype='object')
The length of log is 92094816
###########################################
Number of unique values in user_id: 297915
Number of unique values in timestamp: 89044977
Number of unique values in action_type: 4
Number of unique values in item_id: 29498
Number of unique values in source: 8
Number of unique values in user_answer: 4
Number of unique values in platform: 2
None


In [10]:
check_nan(log)

Number of NaN values in column user_id: 0
Number of NaN values in column timestamp: 0
Number of NaN values in column action_type: 0
Number of NaN values in column item_id: 0
Number of NaN values in column source: 0
Number of NaN values in column user_answer: 67976728
Number of NaN values in column platform: 0


In [13]:
print(log['action_type'].unique())
# need to filter the respond type
log = log[log['action_type'] == 'respond']

['enter' 'respond' 'submit' 'quit']


In [15]:
print(count_unique(log, log.columns))
len(log)

Number of unique values in user_id: 296701
Number of unique values in timestamp: 23298298
Number of unique values in action_type: 1
Number of unique values in item_id: 11555
Number of unique values in source: 7
Number of unique values in user_answer: 4
Number of unique values in platform: 2
None


24118088

In [16]:
92094816-67976728

24118088